# 코드 설계

In [1]:
# 다시한번 생각해 볼 문제... 정호야 정신차리자
# 각각의 읽어온 파일을 sheet로 만들어 하나의 csv 파일로 저장할것. 왜 Why?
# 파일명 예시 nowon.csv, jongno.csv

# 과거의 논리구조 정리
# 데이터 이상치 처리 code
    # -9999, -999를 어떠한 값으로 대체해야하는 것에 대한 problem
    # 결측치를 어떻게 처리할 것인가? -> 무슨 값을 입력해 줄 것인가?
        # 1. 그대로 둔다. (100% 분석할 때 이상치로 될 것)
        # 2. None값으로 바꾼다. (연산할때 문제가 되는지 안되는지 확인해 볼 것)
        # 3. Excel에 있는 Fillter로 적합한 값을 찾아낸다. (빈 결측값들이 너무 많음)
        # 4. ... 그 외의 방법들 활용
        
#     pm25 0~8 최고
#          9~15 좋음
#          16~20 양호
#          21~25 보통
#          26~37 나쁨
#          38~50 상당히 나쁨
#          51~75 매우 나쁨
#          76~ 최악

#     pm10 0~15 최고
#          16~30 좋음
#          31~40 양호
#          41~50 보통
#          51~75 나쁨
#          76~100 상당히 나쁨
#          101~150 매우 나쁨
#          151~ 최악

## Data를 가져오기 위한 code
 * 각각의 구로 작업을 할 것이며, 최종적으로는 함수화를 시킬 것입니다.

### Data 가져오기 (노원구)

In [49]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re
import time

start = time.time()
# 데이터 읽기
base_dir = os.getcwd() # 현재 작업 dir 확인

### 각각 구로 작업할 것 (종로구, 노원구) ###
# 데이터가 있는 경로를 os 모듈을 활용해서 지정해 줄 것
# 후에 data_dir은 함수화를 시킬때, 인자값으로 들어갈 것이다.
data_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구" # 데이터가 들어있는 폴더
working_dir = os.chdir(data_dir) # data_dir로 작업하는 폴더 경로 변경
out_dir = r"C:\Users\User\Documents\data_weatherforcast\노원구_전처리"

files = os.listdir(data_dir) # data_dir의 파일 목록 추출
print("file_list: {}".format(files))

for file in files:
    raw_file = pd.read_csv(file, index_col=0)
    raw_file = raw_file.loc[:, ~raw_file.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제
    
    # 전부 결측치인 columns 제거
    raw_file = raw_file.drop(['co2','vocs'], axis='columns')
    # 데이터 이상치 처리 code -> Not a Number로 변경
    raw_file = raw_file.replace([-999,-9999],[np.nan, np.nan])
    
    ### WHO 기준 label 매기기 code ###
    # 빈 column 두 개 생성
    raw_file['pm10_label'] = None
    raw_file['pm25_label'] = None
    
    # labeling code
    if raw_file['pm10'] is not None:
        excellent = raw_file.loc[raw_file['pm10']<=15,:].index
        raw_file['pm10_label'][excellent] = '최고'
        
        good = raw_file.loc[raw_file['pm10']>=16,:].index
        raw_file['pm10_label'][good] = '좋음'
        
        yangho = raw_file.loc[raw_file['pm10']>=31,:].index
        raw_file['pm10_label'][yangho] = '양호'
        
        soso = raw_file.loc[raw_file['pm10']>=41,:].index
        raw_file['pm10_label'][soso] = '보통'
        
        bad = raw_file.loc[raw_file['pm10']>=51,:].index
        raw_file['pm10_label'][bad] = '나쁨'
        
        very_bad = raw_file.loc[raw_file['pm10']>=76,:].index
        raw_file['pm10_label'][very_bad] = '상당히 나쁨'
        
        terrible = raw_file.loc[raw_file['pm10']>=101,:].index
        raw_file['pm10_label'][terrible] = '매우 나쁨'
        
        worst = raw_file.loc[raw_file['pm10']>=151,:].index
        raw_file['pm10_label'][worst] = '최악'
        
    if raw_file['pm25'] is not None:
        excellent = raw_file.loc[raw_file['pm25']<=8,:].index
        raw_file['pm25_label'][excellent] = '최고'
        
        good = raw_file.loc[raw_file['pm25']>=9,:].index
        raw_file['pm25_label'][good] = '좋음'
        
        yangho = raw_file.loc[raw_file['pm25']>=16,:].index
        raw_file['pm25_label'][yangho] = '양호'
        
        soso = raw_file.loc[raw_file['pm25']>=21,:].index
        raw_file['pm25_label'][soso] = '보통'
        
        bad = raw_file.loc[raw_file['pm25']>=26,:].index
        raw_file['pm25_label'][bad] = '나쁨'
        
        very_bad = raw_file.loc[raw_file['pm25']>=38,:].index
        raw_file['pm25_label'][very_bad] = '상당히 나쁨'
        
        terrible = raw_file.loc[raw_file['pm25']>=51,:].index
        raw_file['pm25_label'][terrible] = '매우 나쁨'
        
        worst = raw_file.loc[raw_file['pm25']>=76,:].index
        raw_file['pm25_label'][worst] = '최악'
        
    # date type 변경 code
    # 작성해야함
    
    print("\n")
    print("="*25,file,"="*25)
    print(type(raw_file)) # DataFrame 형태
    
    # column 이름 변경 code
    raw_file.rename(columns={
        'serial': '측정기 고유번호',
        'flag': '실외 측정기 구분',
        'pm10': '미세먼지(PM-10)',
#         'co2': '이산화탄소 농도(ppm)',
#         'vocs': '휘발성유기화합물 농도',
        'noise': '소음(db)',
        'temp': '온도(℃)',
        'humi': '습도(%)',
        'pm25': '초미세먼지(PM-2.5)',
    }, inplace=True)
    
#     print(raw_file.columns) # columns name 확인
#     print(raw_file.describe())
#     print(raw_file['pm10_label'])
#     print(raw_file['pm25_label'])
    print(file,"작업완료")
    
    # 바뀐 dataframe을 새로운 폴더의 같은 이름으로 to_csv 하는 code
    # 작성해야함
print("time: ", time.time() - start)

file_list: ['V10O1610102.csv', 'V10O1610200.csv', 'V10O1610293.csv', 'V10O1610297.csv', 'V10O1610312.csv', 'V10O1610351.csv', 'V10O1610356.csv', 'V10O1610376.csv', 'V10O1610610.csv', 'V10O1610616.csv', 'V10O1610629.csv', 'V10O1610630.csv', 'V10O1610642.csv', 'V10O1610643.csv', 'V10O1611097.csv', 'V10O1611100.csv', 'V10O1611102.csv', 'V10O1611104.csv', 'V10O1611150.csv', 'V10O1611229.csv', 'V10O1611652.csv', 'V10O1612126.csv']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D



========================= V10O1610102.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610102.csv 작업완료


========================= V10O1610200.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610200.csv 작업완료


========================= V10O1610293.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610293.csv 작업완료


========================= V10O1610297.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610297.csv 작업완료


========================= V10O1610312.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610312.csv 작업완료


========================= V10O1610351.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610351.csv 작업완료


========================= V10O1610356.csv =========================
<class 'pandas.core.frame.DataFrame'>
V10O1610356.csv 작업완료


========================= V10O1610376.csv =========================
<class 'pandas.core.frame.D

### Data 불러오기 (종로구)

In [48]:
# # import moduel
# import pandas as pd
# import os 
# from os import listdir
# import re

# # 데이터 읽기
# base_dir = os.getcwd() # 현재 작업 dir 확인

# ### 각각 구로 작업할 것 (종로구, 노원구) ###
# # 데이터가 있는 경로를 os 모듈을 활용해서 지정해 줄 것
# # 후에 data_dir은 함수화를 시킬때, 인자값으로 들어갈 것이다.
# data_dir = r"C:\Users\User\Documents\data_weatherforcast\종로구" # 데이터가 들어있는 폴더
# working_dir = os.chdir(data_dir) # data_dir로 작업하는 폴더 경로 변경

# files = os.listdir(data_dir) # data_dir의 파일 목록 추출
# print("file_list: {}".format(files))

# for file in files:
#     raw_file = pd.read_csv(file, index_col=0)
#     raw_file = raw_file.loc[:, ~raw_file.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제
#     print("\n")
#     print("="*25,file,"="*25)
#     print(type(raw_file)) # DataFrame 형태
# #     print(raw_file['pm25'].value_counts()) # 제대로 작동하지 않는다.
    
#     # column 이름 변경 code
#     raw_file.rename(columns={
#         'serial': '측정기 고유번호',
#         'flag': '실외 측정기 구분',
#         'pm10': '미세먼지 PM-10',
#         'co2': '이산화탄소 농도(ppm)',
#         'vocs': '휘발성유기화합물 농도',
#         'noise': '소음(db)',
#         'temp': '온도(℃)',
#         'humi': '습도(%)',
#         'pm25': '미세먼지 PM-2.5',
#     }, inplace=True)
#     print(raw_file.columns) # columns name 확인
#     print(raw_file.describe()) # 이상치 때문에 제대로 된 값이 나오지 않음
    
#     ########## 이후 논리 구조 작성 ##########
    
#     # 데이터 이상치 처리 code
#     # -9999, -999를 어떠한 값으로 대체해야하는 것에 대한 problem
#     # 결측치를 어떻게 처리할 것인가? -> 무슨 값을 입력해 줄 것인가?
#         # 1. 그대로 둔다. (100% 분석할 때 이상치로 될 것)
#         # 2. None값으로 바꾼다. (연산할때 문제가 되는지 안되는지 확인해 볼 것)
#         # 3. Excel에 있는 Fillter로 적합한 값을 찾아낸다. (빈 결측값들이 너무 많음)
#         # 4. ... 그 외의 방법들 활용
#     # 이상치 처리 후, 아래 코드 살려서 다시 한번 describe() 하기! 위에 사용하는 code는 삭제할 것
# #     print(raw_file.describe())

#     # WHO와 기상청의 공식 미세먼지 기준을 통해서 Label column을 새로 생성해서 labeling 하는 code
    
#     # 1년간의 미세먼지 수치 변화를 알아보기 위해 전체 데이터를 하나로 합치는 code
    
#     # 파일명 변경..? 불가능! 
#     # (같은 동네에 여러대가 설치되어 있는 경우가 많아서 독립된 값이 측정기고유값밖에 없습니다...)
#     # 하지만 설치된 행정동이나 길 이름을 +_행정동.csv 형식으로 만들 수 있다.
    
#     # 바뀐 dataframe을 새로운 폴더의 같은 이름으로 to_csv 하기 


## 데이터 EDA를 위한 코드 작성

## merge 코드 작성

In [47]:
import numpy as np
import pandas as pd

# sibal